## Polifonia [Patterns Knowledge Graph](https://github.com/polifonia-project/patterns-knowledge-graph) (KG) ingest pipeline. Step 2: Data processing.

In [97]:
# imports

import numpy as np
import pandas as pd
import pickle

This is step 2 of a 2-step data preprocessing pipeline. Step 1 can be found in ```./patters_kg_data_extraction.ipynb``` notebook and must be applied to input corpus before running this notebook.

After running ```patters_kg_data_extraction.ipynb```, point 'kg_data_dir' path in the cell below to ```/[corpus name]/kg_pipeline_input_data``` dir (which contains files outputted by the first preprocessing step).

NOTE: This notebook must be run in full for each individual pattern length. Pattern length is set via 'n' variable in cell below. E.g.: In our sample data below, ```../FoNN/mtc_ann_corpus/kg_pipeline_input_data``` contains patterns of 4, 5, and 6 elements in length outputted by ```patters_kg_data_extraction.ipynb```. Accordingly, users must run this notebook in three passes, one for each of n=4, n=5 and n=6.

Output of this notebook is written to ```../FoNN/mtc_ann_corpus/kg_pipeline_output_data```.
To create a KG from this output data, please see Polifonia [Patterns Knowledge Graph](https://github.com/polifonia-project/patterns-knowledge-graph) repo for further information.


In [98]:
# Load pattern occurrences matrix from kg_pipeline_input_data directory:

kg_data_in_dir = '../mtc_ann_corpus/kg_pipeline_input_data'
n = 4  # set pattern length under investigation
in_file = f'{n}gram_patterns.pkl'
in_path = kg_data_in_dir + '/' + in_file
pattern_occurrences = pd.read_pickle(in_path)
# print number of patterns in input matrices
print(f"{len(pattern_occurrences)} patterns extracted of length {n}")


5901 patterns extracted of length 6


In [99]:
# filter pattern occurrences matrix, retaining patterns which occur twice or more
occurrence_counts = pattern_occurrences.astype('Sparse[int16, 0]')
occurrence_counts.fillna(value=0, inplace=True)
occurrence_counts['freq'] = occurrence_counts.sum(axis=1)
filtered_pattern_occurrences = occurrence_counts[occurrence_counts['freq'] >= 2]
# print number of patterns retained after filtration
print(f"{len(filtered_pattern_occurrences)} patterns retained after filtering.")


3948 patterns retained after filtering.


In [100]:
# change index col type from numpy array to tuples
patterns_reformatted = [tuple(i) for i in filtered_pattern_occurrences.index]
sample_output = patterns_reformatted[:3]
print(f"Sample output:")
print(sample_output)
filtered_patterns = set(patterns_reformatted)


Sample output:
[(1, 1, 1, 1, 1, 3), (1, 1, 1, 1, 2, 1), (1, 1, 1, 1, 2, 7)]


In [101]:
# read locations dict outputted by step 1 of pipeline (patterns_kg_data_extraction.ipynb)
locations_filename = f'{n}gram_locations.pkl'
locations_in_path = kg_data_in_dir + '/' + locations_filename
with open(locations_in_path, 'rb') as locations_raw:
    locations = pickle.load(locations_raw)


In [102]:
# extract tune identifiers from file names in locations data file:
tune_ids = [tune for tune in locations]


In [103]:
# filter locations dict to retain only data for patterns which occur twice or more, to match the pattern occurrences data.

filtered_locations = {}
for tune in locations:
    # read value, which holds a dict of patterns and locations
    pattern_locations = locations[tune]
    # filter
    filtered = {pattern: pattern_locations[pattern] for pattern in pattern_locations if pattern in filtered_patterns}
    filtered_locations[tune] = filtered

# check output
locations_out = dict(zip(tune_ids, list(filtered_locations.values())))
print('Sample output for single tune:')
for k, v in list(locations_out.items())[:1]:
    print(f"identifiers: {k}:\npattern locations: {v}")


Sample output for single tune:
identifiers: 3822:
pattern locations: {(1, 3, 1, 5, 3, 3): [0, 16], (3, 1, 5, 3, 3, 5): [1, 17], (1, 5, 3, 3, 5, 2): [2, 18], (5, 3, 3, 5, 2, 5): [3, 19], (3, 3, 5, 2, 5, 3): [4, 20], (3, 5, 2, 5, 3, 1): [5, 21], (5, 2, 5, 3, 1, 6): [6, 22], (2, 5, 3, 1, 6, 3): [7, 23, 39, 55], (5, 3, 1, 6, 3, 3): [8, 24, 40, 56], (3, 1, 6, 3, 3, 5): [9, 25, 41, 57, 73, 89], (1, 6, 3, 3, 5, 1): [10, 26, 42, 58, 74, 90], (6, 3, 3, 5, 1, 1): [11, 27, 43, 59, 75, 91], (3, 3, 5, 1, 1, 3): [12], (3, 5, 1, 1, 3, 1): [13], (5, 1, 1, 3, 1, 5): [14], (1, 1, 3, 1, 5, 3): [15], (3, 3, 5, 1, 1, 2): [28, 44], (3, 5, 1, 1, 2, 5): [29, 45], (5, 1, 1, 2, 5, 3): [30, 46], (1, 1, 2, 5, 3, 6): [31, 47], (1, 2, 5, 3, 6, 3): [32, 48], (2, 5, 3, 6, 3, 1): [33, 49], (5, 3, 6, 3, 1, 2): [34, 50], (3, 6, 3, 1, 2, 5): [35, 51], (6, 3, 1, 2, 5, 3): [36, 52], (3, 1, 2, 5, 3, 1): [37, 53], (1, 2, 5, 3, 1, 6): [38, 54], (3, 3, 5, 1, 1, 4): [60, 76], (3, 5, 1, 1, 4, 1): [61, 77], (5, 1, 1, 4, 1, 3): [6

In [104]:
# calculate tune lengths (length of feature sequences):

import os

def calculate_tune_lengths(target_dir):
    # calculate length of all feature sequences in input corpus
    tune_durations = []
    for file_name in os.listdir(target_dir):
        if file_name.endswith('.csv'):
            file_path = f"{target_dir}/{file_name}"
            with open(file_path) as content:
                counter = len(content.readlines()) + 1
                tune_durations.append(counter)
    # store in dict per tune id numbers (keys): tune durations (vals)
    tune_durations = dict(zip(tune_ids, tune_durations))

    return tune_durations

# run:
feat_seq_path = '../mtc_ann_corpus/feature_sequence_data/duration_weighted'
tune_lengths = calculate_tune_lengths(feat_seq_path)


In [105]:
import csv
# Extract diatonic scale degree sequences from csv feature sequence files for all tunes
# NOTE: If investigating a different musical feature, please specify below via 'feature' variable.
# The full list of FoNN features is available in ../README.md
feature = 'diatonic_scale_degree'
filenames = [filename[:-4] for filename in os.listdir(feat_seq_path) if filename.endswith('.csv')]
inpaths = [f"{feat_seq_path}/{filename}" for filename in os.listdir(feat_seq_path) if filename.endswith(".csv")]
# identify csv col names
with open(inpaths[0]) as testfile:
    csv_reader = csv.reader(testfile, delimiter=',')
    cols = next(csv_reader)
    colsmap = {col_name: i for i, col_name in enumerate(cols)}
    assert feature in colsmap
# identify index of target column
target_col_idx = colsmap[feature]
# extract target column from all tunes to list
numeric_data = [
    np.genfromtxt(
        path,
        dtype=str,
        delimiter=',',
        usecols=target_col_idx,
            skip_header=1)
        for path in inpaths
    ]
# convert feature sequence data from numeric to string
formatted_feat_seq_data = []
for i in numeric_data:
    numeric = []
    for j in i:
        num = int(float(j)) if j != '' else 0
        numeric.append(num)
    formatted_feat_seq_data.append(numeric)
# store in dict per tune id numbers (keys): feature sequence data (vals)
feat_seq_data =dict(zip(tune_ids, formatted_feat_seq_data))

# confirm order of dict items matches 'locations' variable above
print(filenames == list(locations))
# check output
for k, v in list(feat_seq_data.items())[:1]:
    print(f"identifiers: {k}: \n data ({' '.join(feature.split('_'))} feature sequence values): {v}")


True
identifiers: 3822: 
 data (diatonic scale degree feature sequence values): [1, 3, 1, 5, 3, 3, 5, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 3, 1, 5, 3, 3, 5, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 2, 5, 3, 6, 3, 1, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 2, 5, 3, 6, 3, 1, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 4, 1, 3, 1, 4, 1, 3, 2, 3, 1, 6, 3, 3, 5, 1, 1, 4, 1, 3, 1, 4, 1, 3, 2, 3, 1, 6, 3, 3, 5, 1, 1]


In [106]:
# Setup namedtuple to store output for KG setup:
from typing import NamedTuple
KG_Data = NamedTuple('KG_Data', [
    ('identifiers', str),           # tune id number
    ('feature', str),               # name of musical feature under investigation
    ('level', str),                 # level of granularity of input data. Must be manually defined below in KG_Data instance.
    ('n_vals', tuple),              # range of n-values (i.e.: pattern lengths) under investigation. Must be manually defined below in KG_Data instance.
    ('duration', int),              # length of tune feature sequence
    ('pattern_locations', dict),    # dict of patterns and their locations within the feature sequence
    ('data', list)                  # feature sequence data content
])

# to illustrate and test output mappings,
# use tune id numbers (common keys) to look up values of each of the above NamedTuple data fields from dicts created in celss above
print("Test output mappings for single tune:")
for tune_id_num in tune_ids:
    print(f"identifiers: {tune_id_num}")
    print(f"data: {feat_seq_data[tune_id_num]}")
    print(f"duration: {tune_lengths[tune_id_num]}")
    print(f"pattern_locations: {locations_out[tune_id_num]}")
    print('\n')

Test output mappings for single tune:
identifiers: 3822
data: [1, 3, 1, 5, 3, 3, 5, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 3, 1, 5, 3, 3, 5, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 2, 5, 3, 6, 3, 1, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 2, 5, 3, 6, 3, 1, 2, 5, 3, 1, 6, 3, 3, 5, 1, 1, 4, 1, 3, 1, 4, 1, 3, 2, 3, 1, 6, 3, 3, 5, 1, 1, 4, 1, 3, 1, 4, 1, 3, 2, 3, 1, 6, 3, 3, 5, 1, 1]
duration: 99
pattern_locations: {(1, 3, 1, 5, 3, 3): [0, 16], (3, 1, 5, 3, 3, 5): [1, 17], (1, 5, 3, 3, 5, 2): [2, 18], (5, 3, 3, 5, 2, 5): [3, 19], (3, 3, 5, 2, 5, 3): [4, 20], (3, 5, 2, 5, 3, 1): [5, 21], (5, 2, 5, 3, 1, 6): [6, 22], (2, 5, 3, 1, 6, 3): [7, 23, 39, 55], (5, 3, 1, 6, 3, 3): [8, 24, 40, 56], (3, 1, 6, 3, 3, 5): [9, 25, 41, 57, 73, 89], (1, 6, 3, 3, 5, 1): [10, 26, 42, 58, 74, 90], (6, 3, 3, 5, 1, 1): [11, 27, 43, 59, 75, 91], (3, 3, 5, 1, 1, 3): [12], (3, 5, 1, 1, 3, 1): [13], (5, 1, 1, 3, 1, 5): [14], (1, 1, 3, 1, 5, 3): [15], (3, 3, 5, 1, 1, 2): [28, 44], (3, 5, 1, 1, 2, 5): [29, 45], (5, 1, 1, 2, 5, 3): [30, 46], (1, 1

In [107]:
# populate output KG_Data NamedTuple instances, one for each tune in corpus
output = []
for tune_id_num in tune_ids:
    tune_data = KG_Data(
        identifiers= tune_id_num,
        feature=feature,
        level='duration-weighted note-level',
        data=feat_seq_data[tune_id_num],
        n_vals=(4, 5, 6),
        duration=tune_lengths[tune_id_num],
        pattern_locations=locations_out[tune_id_num])
    output.append(tune_data)

print("Test output:")
print(output[0])
print(len(output))

Test output:
KG_Data(identifiers='3822', feature='diatonic_scale_degree', level='accent-level', n_vals=(4, 5, 6), duration=99, pattern_locations={(1, 3, 1, 5, 3, 3): [0, 16], (3, 1, 5, 3, 3, 5): [1, 17], (1, 5, 3, 3, 5, 2): [2, 18], (5, 3, 3, 5, 2, 5): [3, 19], (3, 3, 5, 2, 5, 3): [4, 20], (3, 5, 2, 5, 3, 1): [5, 21], (5, 2, 5, 3, 1, 6): [6, 22], (2, 5, 3, 1, 6, 3): [7, 23, 39, 55], (5, 3, 1, 6, 3, 3): [8, 24, 40, 56], (3, 1, 6, 3, 3, 5): [9, 25, 41, 57, 73, 89], (1, 6, 3, 3, 5, 1): [10, 26, 42, 58, 74, 90], (6, 3, 3, 5, 1, 1): [11, 27, 43, 59, 75, 91], (3, 3, 5, 1, 1, 3): [12], (3, 5, 1, 1, 3, 1): [13], (5, 1, 1, 3, 1, 5): [14], (1, 1, 3, 1, 5, 3): [15], (3, 3, 5, 1, 1, 2): [28, 44], (3, 5, 1, 1, 2, 5): [29, 45], (5, 1, 1, 2, 5, 3): [30, 46], (1, 1, 2, 5, 3, 6): [31, 47], (1, 2, 5, 3, 6, 3): [32, 48], (2, 5, 3, 6, 3, 1): [33, 49], (5, 3, 6, 3, 1, 2): [34, 50], (3, 6, 3, 1, 2, 5): [35, 51], (6, 3, 1, 2, 5, 3): [36, 52], (3, 1, 2, 5, 3, 1): [37, 53], (1, 2, 5, 3, 1, 6): [38, 54], (3, 3,

In [ ]:
# write KG_Data objects to disc
output_filename = f'{n}gram_kg_data.pkl'
out_dir = '../mtc_ann_corpus/kg_pipeline_output_data'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_path = out_dir + '/' + output_filename
with open(out_path, 'wb') as f_out:
    pickle.dump(output, f_out)
